[Node 14: ctypes](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node14.html)

Navigation:

**Next:** [SWIG](node15.ipynb) **Up:** [SWIG](node15.ipynb) **Previous:** [SWIG](node15.ipynb)

## ctypes
Using the Python standard module [``ctypes``](https://docs.python.org/3/library/ctypes.html), functions can be called from dynamic libraries without extension modules:
> ctypes is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

The data type difference between Python and C is bridged by some ``c_*`` data types, e.g. ``c_short: int/long``, ``c_double: double``, ``c_char_p: str``. An overview of the data types can be found at [hier](https://docs.python.org/3/library/ctypes.html#fundamental-data-types).

In [ ]:
from ctypes import *
i = c_int()
s = c_wchar_p('Hello World!') # corresponding C type: wchar_t * (NUL terminated)
type(s)

In [ ]:
s.value

In [ ]:
i.value

Pointers can also be processed:

In [ ]:
i  = c_int(42)
pi = pointer(i)
pi

In [ ]:
pi.contents.value

``ctypes`` is used to load and call functions from C dynamic libraries. First the library must be loaded in order to then call the C functions:

In [ ]:
libc=cdll.LoadLibrary('libc.so.6')
libc

In [ ]:
dir(libc)

The C functions ``printf`` or ``sleep`` can be loaded dynamically:

In [ ]:
libc.printf

In [ ]:
libc.sleep

In [ ]:
[s for s in dir(libc) if s[:2] != '__' and s[-2:] != '__']

(This list does not include all functions, only those we mentioned earlier. A complete list of all defined functions can be printed with `nm -D`:

In [ ]:
!nm -D `locate libc.so.6 | head -n1`

Argument and return values ​​should be defined using ``argtypes`` and ``restypes``:

In [ ]:
libm=cdll.LoadLibrary('libm.so.6')
cos=libm.cos
cos

In [ ]:
# throws an error:
cos(3.14159265)

In [ ]:
# returns wrong result. By default functions are assumed to return the C int type.
cos(c_double(3.14159265))

In [ ]:
# returns wrong result
cos(c_double(0.0))

In [ ]:
# returns wrong result
cos(c_double(0.1))

In [ ]:
# need to define the call and return parameter types
cos.argtypes=[c_double]
cos.restype=c_double
# now it works
cos(c_double(0.0))

In [ ]:
cos(3.14159265)

Functions that write to a buffer are called like this (example here: [gethostname](http://man7.org/linux/man-pages/man2/gethostname.2.html)):

In [ ]:
libc = cdll.LoadLibrary('libc.so.6')
gethostname = libc.gethostname
gethostname.argtypes = [c_char * 255, c_uint]
gethostname.restype  = c_int
# create the char buffer and call the function
buf = create_string_buffer(255)
gethostname(buf, 3) # too short to hold hostname

In [ ]:
buf.value # is truncated

In [ ]:
gethostname(buf, 30) # more space (our buffer has actually 255 chars)

In [ ]:
buf.value